In [11]:
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType

In [12]:
spark = SparkSession.builder.master('local').appName('MiNADZD').getOrCreate()

df = spark.read.csv('./creditcard.csv', header=True, inferSchema=True)
df = df.withColumn('Class', df['Class'].cast(DoubleType()))